# Flux and magnitude calculations

*J. Runnoe* <br>
*September, 2023*

This notebook will take you through the steps to calculate flux unit conversions and determine magnitudes in different bands. It makes use of the [WFC3 tools](https://github.com/spacetelescope/WFC3Library/tree/master/notebooks), especially the [flux conversion tool notebook](https://github.com/spacetelescope/WFC3Library/blob/main/notebooks/flux_conversion_tool/flux_conversion_tool.ipynb).

---
## Contents
* [Setup](#setup)
    * [Filter Response Curves](#filters)
* [Exercises](#exercises)
    * [Pretty Plots](#plotting)
    * [Flux Unit Conversions](#fluxconv)
    * [Filter Response Curves](#magfromspec)
        * [Define a Source](#sourcedef)
        * [Visualize the Filters](#visualize)
* [Summary](#summary)

---
## Setup <a class="anchor" id="setup"></a>

If you have not set up the `astr8060` environment in `conda` yet, go back and start with the Python Setup notebook.

We will need the `synphot` and `stsynphot` packages:

1. Activate your class Python environment: <br>
    `$ conda activate astr8060` <br>
    
2. Create a new python environment for this class: <br>
    `$ conda install synphot -c conda-forge` <br>

3. Installation isntructions for `stsynphot` are [here](https://stsynphot.readthedocs.io/en/latest/). The following worked for me: <br>
    `$ pip install stsynphot` <br>


#### Filter Response Curves <a class="anchor" id="filters"></a>

In [178]:
# make a filter directory
# do not add this to Github!
import os
from pathlib import Path
fltr_path = '/Users/runnojc1/Dropbox/Research/teaching/F2023/ASTR8060/work/py/filters/'
Path(fltr_path).mkdir(exist_ok=True)
os.chdir(fltr_path)
print("Current directory is: "+os.getcwd())

Current directory is: /Users/runnojc1/Dropbox/Research/teaching/F2023/ASTR8060/work/py/filters


The [SVO website](http://svo2.cab.inta-csic.es/theory/fps/) has filter response curves for many different instruments. Download the following and put them in your filter directory:

[U filter](http://svo2.cab.inta-csic.es/theory/fps/getdata.php?format=ascii&id=Generic/Bessell.U) <br>
[B filter](http://svo2.cab.inta-csic.es/theory/fps/getdata.php?format=ascii&id=Generic/Bessell.B) <br>
[V filter](http://svo2.cab.inta-csic.es/theory/fps/getdata.php?format=ascii&id=Generic/Bessell.V) <br>
[R filter](http://svo2.cab.inta-csic.es/theory/fps/getdata.php?format=ascii&id=Generic/Bessell.R) <br>
[I filter](http://svo2.cab.inta-csic.es/theory/fps/getdata.php?format=ascii&id=Generic/Bessell.I) <br>

---
## Exercises  <a class="anchor" id="exercises"></a>

In [179]:
# import block
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import synphot as syn
from synphot import SourceSpectrum, Observation
from synphot.models import BlackBody1D, PowerLawFlux1D
from synphot.units import convert_flux

from astropy import units as u
from synphot import units as su

from scipy import interpolate, integrate

### Pretty plots <a class="anchor" id="plotting"></a>

It is good practice to make production-quality plots right off the bat. To do this, I include the matplotlib defaults in the following cell.

To use these without executing them in every notebook you write, you can put them in your `matplotlibrc` file. It is usually located in `~/.matplotlib/matplotlibrc`. Note that if you decide to move these to your `matplotlibrc` file, the latex line should be set with `text.latex.preamble=\usepackage{amsmath}`.

In [180]:
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rc('text', usetex=True)
mpl.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"
plt.rc('xtick.major',pad=2)
plt.rc('ytick.major',pad=2)
plt.rc('xtick.minor',pad=2)
plt.rc('ytick.minor',pad=2)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font',size=10,weight='bold')
plt.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['figure.dpi'] = 300

Edit your `.matplotlib/matplotlibrc` file to save your default plot settings.

### Flux Unit Conversions  <a class="anchor" id="fluxconv"></a>

We will use the example from class to practice unit conversions because it is easy to check when things are working. The source we used had a flux density of $3\times10^{-14}$ erg s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}$ at 5000$\unicode[serif]{xC5}.$

To do unit conversions, use the [`synphot.units.convert_flux`](https://synphot.readthedocs.io/en/latest/api/synphot.units.convert_flux.html#synphot.units.convert_flux) convenience function.

It is worth noting that [`astropy`](https://docs.astropy.org/en/stable/units/) and [`synphot`](https://synphot.readthedocs.io/en/latest/synphot/units.html) have similar but not identical units available. A notable difference is that `synphot` has the `PHOTLAM` unit, which is photons s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}.$ I usually just use `astropy` units, but for flux conversion tasks like this I also find it useful to load the `synphot` units.

Useful resources:<br>
[List of available synphot flux units](https://synphot.readthedocs.io/en/latest/synphot/units.html) <br>
[Example flux unit conversion](http://stsdas.stsci.edu/astropy_synphot/synphot/units.html) <br>


1. Define a variable with $f_{\lambda} = 3\times10^{-14}$ erg s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}$ at 5000$\unicode[serif]{xC5}.$

In [181]:
# your answer


<details>

<summary>Solution</summary>

```python
f_lam_cgs = 3e-14*u.erg*(u.s**-1)*(u.cm**-2)*(u.Angstrom**-1)
f_lam_cgs
```
</details>

2. How many erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$ is this?

In [182]:
# your answer


<details>

<summary>Solution</summary>

```python
f_nu_cgs = su.convert_flux(5000.,f_lam_cgs,u.erg*(u.s**-1)*(u.cm**-2)*(u.Hz**-1))
f_nu_cgs
```
</details>

3. How many W m$^{-2}$ Hz$^{-1}$ is this?

In [183]:
# your answer


<details>

<summary>Solution</summary>

```python
f_nu_si = su.convert_flux(5000.,f_lam_cgs,u.W/(u.m**2)/u.Hz)
f_nu_si
```
</details>

4. How many Jansky is this?

In [184]:
# your answer


<details>

<summary>Solution</summary>

```python
f_nu_jy = su.convert_flux(5000.,f_lam_cgs,u.Jy)
f_nu_jy
```
</details>

5. How many photons s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}$ is this?

In [185]:
# your answer


<details>

<summary>Solution</summary>

```python
f_lam_ph = su.convert_flux(5000.,f_lam_cgs,su.PHOTLAM)
f_lam_ph
```
</details>

### Filter Response Curves  <a class="anchor" id="magfromspec"></a>

#### Define a Source  <a class="anchor" id="sourcedef"></a>

Make a [SourceSpectrum](https://synphot.readthedocs.io/en/latest/synphot/spectrum.html) object of Vega and make a plot of the optical spectrum.

You can find an example Vega SourceSpectrum setup [here](https://github.com/spacetelescope/WFC3Library/blob/2ddfbba6f1aae24fa3a4740ca0a3913ede7add50/notebooks/flux_conversion_tool/flux_conversion_tool.ipynb). 

You should make the plot manually. It will be helpful to know that you can extract the flux and wavelength from a SourceSpectrum with:

```python
wave = spec.waveset
flux = spec(wave)
```

The default units are photons s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}.$ Make your plot in erg s$^{-1}$ cm$^{-2}$ $\unicode[serif]{xC5}^{-1}.$

In [186]:
# code here
# add cells as needed


<details>

<summary>Solution</summary>

```python
# define a spectrum
spec = SourceSpectrum.from_vega() 

fig,ax = plt.subplots(1,figsize=(6,4))
wave = spec.waveset
flux = su.convert_flux(wave,spec(wave),su.FLAM)

ax.plot(wave,flux,'k-')
ax.set_xlim(2000,13000)
ax.set_xlabel('Wavelength [\AA]')
ax.set_ylabel('$f_{\lambda}$ [erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$]')
plt.show()

```
</details>

#### Visualize the Filters  <a class="anchor" id="visualize"></a>

Read in the UBVRI filters that you downloaded and plot them. You should make a single plot with all the filter responses in a different colors.

Overplot the spectrum of Vega with a shared x axis to see where the filters will fall. You can create a separate y axis like this:

```python
fig,ax = plt.subplots(1,figsize=(6,4))
ax.plot(fwave,fresp,'b-')
ax.set_xlabel('Wavelength [\AA]')
ax.set_ylabel('Response')
ax2 = ax.twinx()
ax2.plot(vwave,vflux,'k-')
ax2.set_ylabel('$f_{\lambda}$ [ph s$^{-1}$ cm$^{-2}$ \AA$^{-1}$]')
```


In [187]:
# code here
# add cells as needed


<details>

<summary>Solution</summary>

```python
# grab the filter files and names
filter_files = glob.glob(fltr_path+'G*dat')
filters = [file.split('.')[1] for file in filter_files]

# assign colors to filters so they increase with wavelength
clrs = {'U':'purple','B':'blue','V':'green','R':'orange','I':'red'}

# set up the plot
fig,ax = plt.subplots(1,figsize=(6,4))
ax.set_xlim(2000,10000)
ax.set_ylim(0,1)
ax.set_xlabel('Wavelength [\AA]')
ax.set_ylabel('Response')

ax2 = ax.twinx()
ax2.plot(wave,flux,'k-')
ax2.set_ylabel('$f_{\lambda}$ [erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$]')

for file,filter in zip(filter_files,filters):
    # read in the filter response and plot it
    fwave,fresp = np.genfromtxt(file,unpack=True)
    ax.plot(fwave,fresp,color=clrs[filter],alpha=0.5)

```
</details>

---
## Summary <a class="anchor" id="summary"></a>

At this point, all of you should have:
* Installed `synphot` and `stsynphot` on your computer.
* Downloaded some generic filter response curves and know where to find more.
* Updated your `matplotlibrc` file to default to pretty plots.
* Know how to visualize filter response curves.